In [22]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account


# https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# https://codelabs.developers.google.com/codelabs/cloud-bigquery-python#0

# https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# https://cloud.google.com/python/docs/setup


# https://cloud.google.com/apis/docs/getting-started#enabling_apis



# https://cloud.google.com/bigquery/docs/reference/libraries
# https://cloud.google.com/python/docs/reference/bigquery/latest
# https://gcloud.readthedocs.io/en/latest/bigquery-client.html

In [23]:
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
print(credentials.project_id)

smart-data-ml


In [24]:
projectId='smart-data-ml'
table_id = f"{projectId}.SMartDW.incident"
start_date_query='2020-01-01'

In [28]:
client = bigquery.Client(credentials= credentials,project=projectId)
#client = bigquery.Client(project=projectId)

In [26]:
try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table smart-data-ml.SMartDW.incident already exists.


In [20]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

Imported DateTime: 2023-03-31 19:14:34


In [21]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_id}` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Last Imported DateTime: 2023-03-31 16:55:47
Start Import on update_at of last imported date : 2023-03-31 16:55:47


In [8]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [9]:
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,model.model_name as model,

xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type,
CASE WHEN failure_type IS NULL THEN  0 ELSE 1 END AS is_failure_type,

(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail


,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,company.company_name as company
,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where incident.incident_status_id =4
and incident.updated_at>=%(start_date_param)s

order by incident.incident_datetime desc


"""


In [13]:
print("Create all issues dataframe")

dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()


Create all issues dataframe


In [14]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime','updated_at']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')

df_all['imported_at']=dt_imported


df_all.dropna(inplace=True)

#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1 non-null      int64         
 1   incident_no        1 non-null      object        
 2   severity_id        1 non-null      int64         
 3   severity_name      1 non-null      object        
 4   sla                1 non-null      object        
 5   product_type       1 non-null      object        
 6   brand              1 non-null      object        
 7   model              1 non-null      object        
 8   incident_type      1 non-null      object        
 9   status             1 non-null      object        
 10  service_type       1 non-null      object        
 11  is_failure_type    1 non-null      int64         
 12  count_detail       1 non-null      int64         
 13  open_datetime      1 non-null      datetime64[ns]
 14  close_datetime

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,2524,SR-ES-23-2524,2,Major,8x5 4Hrs Response Time,Software,Microsoft,POWER BI,General Incident,Closed,Request,0,1,2023-03-26 09:00:00,2023-03-27 01:00:00,2023-03-26 09:30:00,2023-03-26 23:00:00,YIP,2023-03-31 16:31:03,2023-03-31 16:47:09.737214


In [15]:
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex




Total  1 Imported closed incident to bigquery successfully


In [35]:
importDate=dt_imported.strftime("%Y-%m-%d")

def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx
sql_query=f""" SELECT * FROM `{table_id}` WHERE imported_at >= '{importDate}' LIMIT 10 """
df=load_data_bq(sql_query)
df

,id,incident_no,severity_id,severity_name,sla,product_type,brand,model,incident_type,status,service_type,is_failure_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,company,updated_at,imported_at
0,2466,SR-ES-23-2466,4,Cosmatic,24x7 4Hrs Response Time,Storage,NetApp,SG5612,General Incident,Closed,Request,0,4,2023-03-17 15:48:00,2023-03-30 23:24:00,2023-03-17 15:53:00,2023-03-30 16:24:00,AIS,2023-03-30 23:29:19,2023-03-31 00:10:05.214696
1,2524,SR-ES-23-2524,2,Major,8x5 4Hrs Response Time,Software,Microsoft,POWER BI,General Incident,Closed,Request,0,1,2023-03-26 09:00:00,2023-03-27 01:00:00,2023-03-26 09:30:00,2023-03-26 23:00:00,YIP,2023-03-31 16:31:03,2023-03-31 16:47:09.737214
2,2573,SR-ES-23-2573,4,Cosmatic,Non MA,Server,HPE,DL320-G6,Fan Failure,Closed,Request,0,1,2023-03-28 09:30:00,2023-03-28 11:00:00,2023-03-28 10:00:00,2023-03-28 11:00:00,TLI,2023-03-30 23:34:44,2023-03-31 00:10:05.214696
3,2567,SR-ES-23-2567,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,FAS8200,General Incident,Closed,Incident,0,4,2023-03-28 18:32:00,2023-03-30 16:46:00,2023-03-28 18:40:00,2023-03-28 18:41:00,AIS,2023-03-30 23:32:37,2023-03-31 00:10:05.214696
4,2575,SR-ES-23-2575,2,Major,24x7 6Hrs Resolution Time,Software,YIP,Incident System Management,Software,Closed,Request,0,1,2023-03-31 12:00:00,2023-03-31 16:53:00,2023-03-31 13:10:00,2023-03-31 16:53:00,YIP,2023-03-31 16:54:39,2023-03-31 16:55:47.105650
